# FPL Data Generator

This notebook scrapes played matches, then generates the required CSV files for the accompanying Tableau dashboards.

In [1]:
#Standard data manipulations
import pandas as pd
import numpy as np

from datetime import date
import sqlite3
import pickle

#Import custom functions
from PythonFunctions.apicallers import *
from PythonFunctions.sqlfunctions import *
from PythonFunctions.matchscrapers import *

#Suppress warnings from showing
import warnings
warnings.filterwarnings('ignore')

#Connect to the SQL database
conn = sqlite3.connect('Data/20_21fpl.db')
#Instantiate a cursor
c = conn.cursor()

Work out which matches we ought to have data for, which aren't present in the database.

In [2]:
to_run = suggested_match_ids()
print(to_run)

[59266, 59267, 59268, 59269, 59270, 59271, 59272, 59273, 59274, 59275]


Update data for these matches.

In [3]:
CoreDataUpdater(to_run, c, conn)


FAILURE: Match 59266 failed to scrape - may not have been played yet


FAILURE: Match 59267 failed to scrape - may not have been played yet


FAILURE: Match 59268 failed to scrape - may not have been played yet


FAILURE: Match 59269 failed to scrape - may not have been played yet


FAILURE: Match 59270 failed to scrape - may not have been played yet


FAILURE: Match 59271 failed to scrape - may not have been played yet


FAILURE: Match 59272 failed to scrape - may not have been played yet


FAILURE: Match 59273 failed to scrape - may not have been played yet

Stored match 0.0 (ID: 59274)
Match 59274 object successfully instantiated
Match 59274 shot detail dataframe successfully created
Match 59274 player detail dataframe successfully created
Match 59274 team detail dataframe successfully created
Match 59274 SQL entries committed
Stored match 0.0 (ID: 59275)
Match 59275 object successfully instantiated
Match 59275 shot detail dataframe successfully created
Match 59275 player detail da

Sweep up matches that the updater might have missed.

In [4]:
MatchSweeper(c, conn)

Attempting to scrape [59266, 59267, 59268, 59269, 59270, 59271, 59272, 59273]
Stored match 0.0 (ID: 59266)
Match 59266 object successfully instantiated
Match 59266 shot detail dataframe successfully created
Match 59266 player detail dataframe successfully created
Match 59266 team detail dataframe successfully created
Match 59266 SQL entries committed
Stored match 0.0 (ID: 59267)
Match 59267 object successfully instantiated
Match 59267 shot detail dataframe successfully created
Match 59267 player detail dataframe successfully created
Match 59267 team detail dataframe successfully created
Match 59267 SQL entries committed
Stored match 0.0 (ID: 59268)
Match 59268 object successfully instantiated
Match 59268 shot detail dataframe successfully created
Match 59268 player detail dataframe successfully created
Match 59268 team detail dataframe successfully created
Match 59268 SQL entries committed
Stored match 0.0 (ID: 59269)
Match 59269 object successfully instantiated
Match 59269 shot detail

# CSV Creation

In [5]:
#Bring in XG functions now (note this is brought in after the new goal data is known)
from PythonFunctions.xg import *

#Generate a player detail dataframe, with the XG stats 
df_pm = pd.merge(df_pm_generator(),
                 api_stat_generator(),
                 on=['MatchID','Player']).drop(['TableIndex','Minutes'], axis=1)

df_tm = df_tm_generator()

In [6]:
df_pm.tail(10)

,MatchID,Player,GameWeek,ForTeam,AgainstTeam,RelativeStrength,Goals,ShotsOnTarget,ShotsInBox,CloseShots,...,Position,Date,BPS,CleanSheet,MinutesPlayed,NetTransfersIn,Points,Price,Saves,SelectedBy
23338,59273,Jayden Bogle,38,Sheffield United,Burnley,0,0,0,0,0,...,DEF,2021-05-23,0,0.0,0.0,53,0,4.3,0.0,7142
23339,59273,Max Lowe,38,Sheffield United,Burnley,0,0,0,0,0,...,DEF,2021-05-23,0,0.0,0.0,9,0,4.2,0.0,4078
23340,59273,Phil Jagielka,38,Sheffield United,Burnley,0,0,0,0,0,...,DEF,2021-05-23,3,0.0,7.0,-128,1,3.9,0.0,43095
23341,59273,Michael Verrips,38,Sheffield United,Burnley,0,0,0,0,0,...,GKP,2021-05-23,0,0.0,0.0,-302,0,4.0,0.0,16745
23342,59273,Kean Bryan,38,Sheffield United,Burnley,0,0,0,0,0,...,DEF,2021-05-23,0,0.0,0.0,-84,0,3.9,0.0,38212
23343,59273,Antwoine Hackford,38,Sheffield United,Burnley,0,0,0,0,0,...,FWD,2021-05-23,0,0.0,0.0,7,0,4.4,0.0,9627
23344,59273,Kyron Gordon,38,Sheffield United,Burnley,0,0,0,0,0,...,DEF,2021-05-23,0,0.0,0.0,41,0,4.0,0.0,1791
23345,59273,Iliman Ndiaye,38,Sheffield United,Burnley,0,0,0,0,0,...,MID,2021-05-23,0,0.0,0.0,3,0,4.5,0.0,231
23346,59273,Zak Brunt,38,Sheffield United,Burnley,0,0,0,0,0,...,MID,2021-05-23,0,0.0,0.0,20,0,4.5,0.0,366
23347,59273,Frankie Maguire,38,Sheffield United,Burnley,0,0,0,0,0,...,MID,2021-05-23,0,0.0,0.0,38,0,4.4,0.0,638


In [7]:
df_tm.tail(10)

,MatchID,ForTeam,AgainstTeam,RelativeStrength,GameWeek,Home,Possession,Goals,ShotsOnTarget,TotalShots,...,ShotsLeft,ShotsRight,ShotsInBoxConceded,CloseShotsConceded,HeadersConceded,ShotsCentreConceded,ShotsLeftConceded,ShotsRightConceded,XG,XGC
750,59269,Leeds United,West Bromwich Albion,0,38,Home,64.4,3,9,17,...,0,0,7,8,3,8,0,0,3.477948,2.181888
751,59269,West Bromwich Albion,Leeds United,0,38,Away,35.6,1,5,14,...,0,0,6,8,1,6,0,0,2.181888,3.477948
752,59270,Leicester City,Tottenham Hotspur,0,38,Home,47.0,2,6,10,...,0,1,8,5,1,5,3,0,2.816824,2.534449
753,59270,Tottenham Hotspur,Leicester City,0,38,Away,53.0,4,4,11,...,3,0,4,6,2,3,0,1,2.534449,2.816824
754,59271,Liverpool,Crystal Palace,2,38,Home,69.6,2,5,19,...,0,4,1,5,0,1,1,1,3.197027,0.829717
755,59271,Crystal Palace,Liverpool,-2,38,Away,30.4,0,4,5,...,1,1,15,7,6,11,0,4,0.829717,3.197027
756,59272,Manchester City,Everton,2,38,Home,67.7,5,11,21,...,3,3,2,3,0,1,1,1,3.559166,1.482105
757,59272,Everton,Manchester City,-2,38,Away,32.3,0,3,8,...,1,1,14,11,3,9,3,3,1.482105,3.559166
758,59273,Sheffield United,Burnley,0,38,Home,43.0,1,3,12,...,2,4,5,4,5,1,0,4,0.968887,1.488896
759,59273,Burnley,Sheffield United,0,38,Away,57.0,0,3,10,...,0,4,7,5,4,2,2,4,1.488896,0.968887


In [8]:
df_matches.drop('TableIndex', axis=1).to_csv('Outputs/fixtures.csv')
df_pm.to_csv('Outputs/player_data.csv')
df_tm.to_csv('Outputs/team_data.csv')

In [9]:
def next_x_games(team, x, df_tm = df_tm):
    
    '''
    returns a dataframe for the next x games for a given team
    '''

    df_temp = df_matches[(df_matches['Team']==team)
              &~(df_matches['MatchID']).isin(list(df_tm['MatchID']))].head(x)
    
    return df_temp[['MatchID','Opposition']]



def prev_x_games(team, x, df_tm = df_tm):
    
    #Create a dataframe with the right team
    return df_tm.loc[df_tm['ForTeam']==team].sort_values('GameWeek', ascending=False).head(x)



def weighted_prev_xg(team, x, final_weight = 0.5, df_tm = df_tm):
    
    df_temp = prev_x_games(team=team, x=x, df_tm=df_tm)
    
    #Calculate weights that we will apply to historical values to get an
    #average that emphasises recent performance
    x = len(df_temp)
    
    df_temp['Weights'] = [1-i*(1-final_weight)/(x-1) for i in range(x)]
    df_temp['Weights'] = df_temp['Weights'].map(lambda x: x/df_temp['Weights'].sum())
    
    df_temp['XG_Weighted'] = df_temp['Weights'] * df_temp['XG']
    df_temp['XGC_Weighted'] = df_temp['Weights'] * df_temp['XGC']
    
    xg = df_temp['XG_Weighted'].sum()
    xgc = df_temp['XGC_Weighted'].sum()

    return {'xg_weighted':xg,
           'xgc_weighted':xgc}


def upcoming_xg_index(team, x_back, x_forward,
                      final_weight = 0.5, forward_weight = 0.25,
                      df_tm = df_tm):
    '''
    Creates a dataframe for a team, which has the XG and XGC of itself and
    upcoming teams, as calculated by the weighted functions
    '''
    
    df_stub = next_x_games(team, x=x_forward, df_tm = df_tm)
    
    team_stats = weighted_prev_xg(team, x_back, final_weight = final_weight)
    df_stub['xg'] = [team_stats['xg_weighted'] for i in range(x_forward)]
    df_stub['xgc'] = [team_stats['xgc_weighted'] for i in range(x_forward)]
    
    df_stub['Opp_xg'] = [weighted_prev_xg(i, x_back,
                                          final_weight = final_weight)['xg_weighted']
                          for i in df_stub['Opposition']]
    
    df_stub['Opp_xgc'] = [weighted_prev_xg(i, x_back,
                                           final_weight = final_weight)['xgc_weighted']
                          for i in df_stub['Opposition']]
        
    df_stub['xg_index'] = df_stub['xg'] * df_stub['Opp_xgc']
    df_stub['xgc_index'] = df_stub['xgc'] * df_stub['Opp_xg']
    
    x = len(df_stub)
    
    df_stub['Weights'] = [1-i*(1-forward_weight)/(x-1) for i in range(x)]
    df_stub['Weights'] = df_stub['Weights'].map(lambda x: x/df_stub['Weights'].sum())
    
    xg_index_avg = (df_stub['Weights'] * df_stub['xg_index']).sum()
    xgc_index_avg = (df_stub['Weights'] * df_stub['xgc_index']).sum()
    
    return xg_index_avg, xgc_index_avg


def upcoming_xg_index_df(x_back = 4, x_forward = 4,
                      final_weight = 0.5, forward_weight = 0.4,
                      df_tm = df_tm):
    
    df_temp = pd.DataFrame(columns = ['Team','XGIndex','XGCIndex'])
    
    for i in df_teams['CommentName']:
        indices = upcoming_xg_index(i, x_back, x_forward,
                                    final_weight, forward_weight, df_tm)
        
        new_row = pd.DataFrame({'Team':[i],
                                'XGIndex':[indices[0]],
                                'XGCIndex':[indices[1]]})
        
        df_temp = pd.concat([df_temp, new_row]).reset_index(drop=True)
        
    return df_temp

In [10]:
df_xg_index = upcoming_xg_index_df()
df_xg_index.to_csv('Outputs/xg_index.csv')

In [11]:
df_xg_index.sort_values('XGIndex', ascending=False)

,Team,XGIndex,XGCIndex
0,Arsenal,0.0,0.0
1,Aston Villa,0.0,0.0
18,West Ham United,0.0,0.0
17,West Bromwich Albion,0.0,0.0
16,Tottenham Hotspur,0.0,0.0
15,Southampton,0.0,0.0
14,Sheffield United,0.0,0.0
13,Newcastle United,0.0,0.0
12,Manchester United,0.0,0.0
11,Manchester City,0.0,0.0


In [12]:
df_xg_index.sort_values('XGCIndex')

,Team,XGIndex,XGCIndex
0,Arsenal,0.0,0.0
17,West Bromwich Albion,0.0,0.0
16,Tottenham Hotspur,0.0,0.0
15,Southampton,0.0,0.0
14,Sheffield United,0.0,0.0
13,Newcastle United,0.0,0.0
12,Manchester United,0.0,0.0
11,Manchester City,0.0,0.0
10,Liverpool,0.0,0.0
9,Leeds United,0.0,0.0


Ta dah.